In [1]:
import nuclio 
import mlrun

In [2]:
mlrun.get_version()

'0.6.2-rc1'

In [17]:
import storey

In [3]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [4]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames
pip install storey==0.4.0

In [5]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [6]:
%nuclio cmd -c pip install v3io

In [7]:
# nuclio: start-code

In [8]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd
import numpy
import os
import math

In [9]:
def rm_inf(data):
    print(data)
    data = [0 if item == math.inf or item == -math.inf else item for item in data ]
    #data = [0 for item in data if item == -math.inf]
    return data

In [10]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    stock_list = []
    # Reading the data from feature service
    service = fs.get_online_feature_service("stocks-vec")
    for key,value in context.sym_to_url.items():
        context.logger.info(key)
        data = service.get([{"symbol": key}])[0]
        data["symbol"] = key
        stock_list.append(data)
    
    data = [rm_inf(list(item.values())) for item in stock_list]
    
    columns = [{'text': key, 'type': 'object'} for key in stock_list[0].keys()]
    
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [11]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-test-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc', 'INTC' : 'intel-corp'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.prices_min_1h",
                "stocks.prices_max_1h",
                "stocks.volumes_min_1h",
                "stocks.volumes_max_1h",
                "stocks.price",
                "stocks.volume",
                "stocks.last_updated",
                "news.sentiments_min_1h",
                "news.sentiments_max_1h",
                "news.sentiment",
                "news.last_reaction"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [12]:
# nuclio: end-code

In [13]:
# test local
init_context(context)
event = ""#nuclio.Event(body='')
df = handler(context,event)

Python> 2021-04-06 11:57:37,216 [info] Initalizing context & constructing featrure vector
Python> 2021-04-06 11:57:37,252 [info] Reading features from feature vector
Python> 2021-04-06 11:57:37,310 [info] GOOGL
Python> 2021-04-06 11:57:37,324 [info] MSFT
Python> 2021-04-06 11:57:37,328 [info] AMZN
Python> 2021-04-06 11:57:37,332 [info] AAPL
Python> 2021-04-06 11:57:37,336 [info] INTC
[-0.3333333333333333, -0.3333333333333333, inf, -inf, inf, -inf, 2219.06005859375, 36701.0, '2021-04-05 15:59:00-04:00', -0.3333333333333333, '2021-04-05 19:53:57', 'GOOGL']
[0.3333333333333333, 0.6666666666666666, inf, -inf, inf, -inf, 249.0, 502948.0, '2021-04-05 15:59:00-04:00', 0.6666666666666666, '2021-04-04 11:00:54', 'MSFT']
[1.0, 1.0, inf, -inf, inf, -inf, 3225.260009765625, 36889.0, '2021-04-05 15:59:00-04:00', 1.0, '2021-04-05 19:38:39', 'AMZN']
[0.6666666666666666, 0.6666666666666666, inf, -inf, inf, -inf, 125.9000015258789, 1287009.0, '2021-04-05 15:59:00-04:00', 0.6666666666666666, '2021-04-05

In [14]:
# test remote deployment
from mlrun import code_to_function
fn = code_to_function('vector_reader',
                      handler='handler')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : 'stocks-test-dani',
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [15]:
addr = fn.deploy(project='stocks-test-dani')

> 2021-04-06 11:57:44,604 [info] Starting remote function deploy
2021-04-06 11:57:44  (info) Deploying function
2021-04-06 11:57:44  (info) Building
2021-04-06 11:57:44  (info) Staging files and preparing base images
2021-04-06 11:57:44  (info) Building processor image
2021-04-06 11:57:46  (info) Build complete
> 2021-04-06 11:57:50,334 [info] function deployed, address=default-tenant.app.dev8.lab.iguazeng.com:31504


In [16]:
!curl {addr}

[{"columns": [{"text": "sentiments_min_1h", "type": "object"}, {"text": "sentiments_max_1h", "type": "object"}, {"text": "prices_min_1h", "type": "object"}, {"text": "prices_max_1h", "type": "object"}, {"text": "volumes_min_1h", "type": "object"}, {"text": "volumes_max_1h", "type": "object"}, {"text": "price", "type": "object"}, {"text": "volume", "type": "object"}, {"text": "last_updated", "type": "object"}, {"text": "sentiment", "type": "object"}, {"text": "last_reaction", "type": "object"}, {"text": "symbol", "type": "object"}], "rows": [[-0.3333333333333333, -0.3333333333333333, 0, 0, 0, 0, 2219.06005859375, 36701.0, "2021-04-05 15:59:00-04:00", -0.3333333333333333, "2021-04-05 19:53:57", "GOOGL"], [0.3333333333333333, 0.6666666666666666, 0, 0, 0, 0, 249.0, 502948.0, "2021-04-05 15:59:00-04:00", 0.6666666666666666, "2021-04-04 11:00:54", "MSFT"], [1.0, 1.0, 0, 0, 0, 0, 3225.260009765625, 36889.0, "2021-04-05 15:59:00-04:00", 1.0, "2021-04-05 19:38:39", "AMZN"], [0.6666666666666666,